In [33]:
import os
from radiomics import featureextractor
import pandas as pd
import glob
import csv
import SimpleITK as sitk
import keras
import numpy as np
import pickle

In [ ]:
paramPath = os.path.join(os.getcwd(), "Params.yaml")
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)
df = pd.read_csv('output_three_classes2.csv')
directories = os.listdir("./dataCorrected")

In [ ]:
df = pd.read_csv('output_three_classes.csv')
directories = os.listdir("./dataCorrected")
for d in directories:
#     image = glob.glob(f"./dataCorrected/{d}/*flair.nii.gz")
#     imagePath = os.path.join(os.getcwd(), image[0])
    image = pickle.load( open( "D:\croppedFiles/%s_images.pkl"%(d), "rb" ) )
    seg = pickle.load( open( "D:\croppedFiles/%s_seg_mask_3ch.pkl"%(d), "rb" ) )
    image = sitk.GetImageFromArray(image[3])
    label = sitk.GetImageFromArray(seg[2])
    result = extractor.execute(image, label)
    keys, values = [], []
#     keys.append('Age')
#     values.append(df[df['BraTS19ID'] == d]['Age'].values[0])
    for key, value in result.items():
        keys.append(key)
        values.append(value)
    keys.append('Survival')
    values.append(df[df['BraTS19ID'] == d]['Survival'].values[0])
    with open("./features/resultseg2.csv", "a") as outfile:
        csvwriter = csv.writer(outfile)
        if d == directories[0]:
            csvwriter.writerow(keys)
        csvwriter.writerow(values)

In [ ]:
df = pd.read_csv('./features/resultseg2.csv')
df.to_csv('./features/outputseg2.csv', index=False)
data = pd.read_csv('./features/outputseg2.csv')
data.head()

In [ ]:
data.columns[:22]
data.columns[21]
data.drop(df.iloc[:, :22], inplace = True, axis = 1)
data.to_csv('./features/data3_seg2.csv')

In [ ]:
import csv
with open('./features/data0_seg2.csv','r') as f1, open('./features/data0_seg1.csv','r') as f2,open('./features/data0_seg0.csv','r') as f3,open('./features/data1_seg2.csv','r') as f4, open('./features/data1_seg1.csv','r') as f5,open('./features/data1_seg0.csv','r') as f6,open('./features/data2_seg2.csv','r') as f7, open('./features/data2_seg1.csv','r') as f8,open('./features/data2_seg0.csv','r') as f9,open('./features/data3_seg0.csv','r') as f10, open('./features/data3_seg1.csv','r') as f11,open('./features/data3_seg2.csv','r') as f12, open('./features/all_data.csv','w') as w:
    writer = csv.writer(w)
    r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12 = csv.reader(f1),csv.reader(f2),csv.reader(f3),csv.reader(f4),csv.reader(f5),csv.reader(f6),csv.reader(f7),csv.reader(f8),csv.reader(f9),csv.reader(f10),csv.reader(f11),csv.reader(f12)
    while True:
        try:
            writer.writerow(next(r1)+next(r2)+next(r3)+next(r4)+next(r5)+next(r6)+next(r7)+next(r8)+next(r9)+next(r10)+next(r11)+next(r12))
        except StopIteration:
            break

In [ ]:
data = pd.read_csv('./features/all_data.csv')

In [ ]:
m = data['Age'].mean()
data['Age'].fillna(value=m, inplace=True)
data['Survival'] = data['Survival'].fillna(0)
data.to_csv('./features/all_data_filled.csv')

In [1]:
from sklearn.preprocessing import StandardScaler
# load data
data = pd.read_csv('./features/from_predictions/top_corr_data_20_c.csv')
# # create scaler
y = data['Survival']
y.fillna(0)
del data['Survival']
scaler = StandardScaler()
# fit and transform in one step
standardized = scaler.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(standardized, y, test_size=0.2, random_state=42)

C:\Users\HassanIslam\anaconda3\envs\FYP\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HassanIslam\anaconda3\envs\FYP\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\HassanIslam\anaconda3\envs\FYP\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


NameError: name 'pd' is not defined

In [7]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate,GridSearchCV
# clf = SVC()
clf = GaussianNB()
# parameters = {
#     "kernel": ["linear","rbf"],
#     "C":[0.1,1,10]
# }
# clf=GridSearchCV(clf,parameters,cv=10,n_jobs=-1,verbose=1)
clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
print(clf.score(X_test, y_test))
confusion_matrix(y_test, y_pred)
# print(clf.best_params_)

R2 score   0.05123768320367883
MSE   141841.62634423078
RMSE   376.6186749807168
MAE   298.8090384615385


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(data)
data2 = pca.transform(standardized)
X_train, X_test, y_train, y_test = train_test_split(data2, y, test_size=0.2)
clf = SVR()


parameters = {
    "kernel": ["linear","rbf"],
    "C":[0.1,1,10],
    "epsilon":[0.1,0.5,1.0]
}
clf=GridSearchCV(clf,parameters,cv=10,n_jobs=-1,verbose=1)

clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
print(f"R2 score   {metrics.r2_score(y_test, y_pred)}")
print(f"MSE   {metrics.mean_squared_error(y_test, y_pred)}")
print(f"RMSE   {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}")
print(f"MAE   {metrics.mean_absolute_error(y_test, y_pred)}")
print(clf.best_params_)

In [ ]:
import xgboost as xgb
pca = PCA()
pca.fit(data)
data2 = pca.transform(standardized)
X_train, X_test, y_train, y_test = train_test_split(data2, y, test_size=0.2)
clf = xgb.XGBRegressor()
clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
print(f"R2 score   {metrics.r2_score(y_test, y_pred)}")
print(f"MSE   {metrics.mean_squared_error(y_test, y_pred)}")
print(f"RMSE   {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}")
print(f"MAE   {metrics.mean_absolute_error(y_test, y_pred)}")

In [5]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
data = pd.read_csv('./from_predictions/top_corr_data_20_c.csv')
train = data[:200]
test = data[200:]
target_train = train['Survival']
train.drop(['Survival'],axis = 1 , inplace = True)
target_test = test['Survival']
test.drop(['Survival'],axis = 1 , inplace = True)
scaler = StandardScaler()
standardized_train = scaler.fit_transform(train)
standardized_test = scaler.fit_transform(test)

C:\Users\HassanIslam\anaconda3\envs\FYP\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [22]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate,GridSearchCV
clf = RandomForestClassifier()
# clf = GaussianNB()
# parameters = {

#     "kernel": ["linear","rbf"],
#     "C":[0.1,1,10]
# }
# clf=GridSearchCV(clf,parameters,cv=10,n_jobs=-1,verbose=1)
clf.fit(standardized_train, target_train.values.ravel())
y_pred = clf.predict(standardized_test)
print(clf.score(standardized_test, target_test))
confusion_matrix(target_test, y_pred)
# print(clf.best_params_)

0.5087719298245614


array([[19, 12,  2],
       [10, 10,  1],
       [ 0,  3,  0]], dtype=int64)

In [ ]:
NN_model = Sequential()
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = standardized_train.shape[1], activation='relu'))
NN_model.add(Dropout(0.4))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.4))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.4))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.4))
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
NN_model.fit(standardized_train, target_train, epochs=500, batch_size=32, validation_split = 0.2)

In [ ]:
from sklearn import metrics
y_pred = NN_model.predict(standardized_test)
print(f"R2 score   {metrics.r2_score(target_test, y_pred)}")
print(f"MSE   {metrics.mean_squared_error(target_test, y_pred)}")
print(f"RMSE   {np.sqrt(metrics.mean_squared_error(target_test, y_pred))}")
print(f"MAE   {metrics.mean_absolute_error(target_test, y_pred)}")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
clf = RandomForestRegressor()
clf.fit(standardized_train, target_train.values.ravel())
y_pred = clf.predict(standardized_test)
print(f"R2 score   {metrics.r2_score(target_test, y_pred)}")
print(f"MSE   {metrics.mean_squared_error(target_test, y_pred)}")
print(f"RMSE   {np.sqrt(metrics.mean_squared_error(target_test, y_pred))}")
print(f"MAE   {metrics.mean_absolute_error(target_test, y_pred)}")

In [ ]:
import xgboost as xgb
clf = xgb.XGBRegressor()
clf.fit(standardized_train, target_train.values.ravel())
y_pred = clf.predict(standardized_test)
print(f"R2 score   {metrics.r2_score(target_test, y_pred)}")
print(f"MSE   {metrics.mean_squared_error(target_test, y_pred)}")
print(f"RMSE   {np.sqrt(metrics.mean_squared_error(target_test, y_pred))}")
print(f"MAE   {metrics.mean_absolute_error(target_test, y_pred)}")

In [30]:
import pandas as pd
data = pd.read_csv('./from_predictions/top_corr_data_20.csv')
data.loc[(data['Survival'].astype(float) < 300), 'Survival'] = 0
data.loc[(data['Survival'].astype(float) > 900), 'Survival'] = 2
data.loc[((data['Survival'].astype(float) >= 300) & (data['Survival'].astype(float) <= 900)), 'Survival'] = 1

In [31]:
data.tail(20)

,original_glcm_Correlation.2,Age,original_glszm_ZoneEntropy.1,original_shape_MinorAxisLength,original_shape_LeastAxisLength.3,original_shape_SurfaceArea.1,original_shape_MinorAxisLength.1,original_glcm_Idn.2,original_shape_Maximum2DDiameterSlice.5,original_glrlm_RunLengthNonUniformity.3,original_glcm_Idmn.2,original_glszm_GrayLevelNonUniformity.5,original_shape_Maximum2DDiameterColumn,original_shape_LeastAxisLength.5,original_shape_Maximum2DDiameterRow.2,original_shape_Maximum2DDiameterRow.3,Survival
237,0.845113,52.687671,7.249129,41.312657,26.789309,17570.79205,55.360350,0.953187,63.639610,15467.939510,0.995279,440.094048,53.160135,41.108674,63.600314,56.515485,0
238,0.896781,74.906849,6.427557,41.598372,32.615202,29759.49347,68.907959,0.971536,77.620873,18020.454100,0.997880,535.692397,65.604878,47.244152,75.059976,73.109507,0
239,0.878663,54.698630,7.368026,50.305947,33.252611,31430.68914,65.885661,0.965793,76.655072,18700.061500,0.997380,279.178806,66.037868,37.335287,75.312682,74.525164,0
240,0.778723,57.345205,4.964117,25.471799,21.056433,14367.89706,45.805649,0.961767,38.078866,2139.258563,0.993304,137.143411,41.617304,26.187365,40.816663,37.589892,1
241,0.842907,58.934247,7.002691,30.992905,21.026664,16843.05378,47.493463,0.963759,40.853396,1381.685003,0.996630,99.601569,32.202484,19.928277,31.400637,31.780497,0
242,0.897719,66.479452,7.012183,40.134393,33.182189,27603.08910,70.506233,0.966161,67.186308,6054.490081,0.997280,602.792395,56.320511,36.519732,68.066144,62.585941,0
243,0.900900,50.501370,6.069945,38.814535,28.953802,14081.86209,47.510716,0.966971,50.477718,20737.870330,0.997416,184.438803,56.080300,28.941122,49.497475,50.039984,2
244,0.913426,57.361644,6.441519,30.980724,21.403710,16083.22995,41.943997,0.963211,42.544095,1092.799289,0.996721,107.157296,33.015148,27.447237,37.656341,36.496575,1
245,0.830659,64.860274,5.923990,33.755143,27.115515,14134.57135,52.212117,0.967272,49.648766,8183.203062,0.997067,160.595604,43.046487,26.606374,49.254441,48.507731,0
246,0.680311,78.745205,6.584125,5.415523,3.738339,21275.54568,52.252214,0.933485,24.413111,43.774612,0.989082,51.040909,23.345235,12.762551,28.071338,5.385165,0


In [ ]:
data.to_csv()